TASK-2

In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, sum, avg, month, year, count, expr

# Initialize Spark session
spark = SparkSession.builder.appName("Sales Aggregation and Feature Engineering").getOrCreate()

# Load the sales dataset
sales_df = spark.read.csv("/workspaces/Project-2-Retail-Sales-Analytics-and-Real-Time-Demand-Forecasting/task2/Online Retail - Online Retail (1).csv", header=True, inferSchema=True)

# Inspect dataset
print("Initial dataset:")
sales_df.show(10)
sales_df.printSchema()

# Ensure `InvoiceDate` is in proper datetime format
sales_df = sales_df.withColumn("InvoiceDate", col("InvoiceDate").cast("timestamp"))

# Calculate revenue for each transaction
sales_df = sales_df.withColumn("RevenueGenerated", col("Quantity") * col("UnitPrice"))

# Extract month and year for analysis
sales_df = sales_df.withColumn("Year", year(col("InvoiceDate"))).withColumn("Month", month(col("InvoiceDate")))

# Total sales per product per month
print("Calculating total sales per product per month...")
total_sales_per_product = sales_df.groupBy("Year", "Month", "StockCode", "Description") \
    .agg(
        sum("Quantity").alias("TotalQuantitySold"),
        sum("RevenueGenerated").alias("TotalRevenueGenerated")
    ) \
    .orderBy("Year", "Month", "StockCode")

total_sales_per_product.show(10)

# Average revenue per customer segment (Country used as proxy for customer segments)
print("Calculating average revenue per customer segment...")
avg_revenue_per_country = sales_df.groupBy("Country") \
    .agg(avg("RevenueGenerated").alias("AvgRevenuePerCountry"))

avg_revenue_per_country.show()

# Seasonal patterns for top-selling products
print("Analyzing seasonal patterns for top-selling products...")
# Identify top-selling products by total sales
top_products = total_sales_per_product.groupBy("StockCode", "Description") \
    .agg(sum("TotalQuantitySold").alias("TotalSales")) \
    .orderBy(col("TotalSales").desc()) \
    .limit(5)  # Top 5 products

top_products.show()

# Seasonal trends for top products
seasonal_trends = total_sales_per_product.join(top_products, ["StockCode", "Description"]) \
    .groupBy("StockCode", "Description", "Month") \
    .agg(sum("TotalQuantitySold").alias("MonthlySales")) \
    .orderBy("StockCode", "Month")

seasonal_trends.show()

# Feature Engineering
print("Feature engineering...")

# Customer Lifetime Value (CLV): Total revenue generated by each customer
clv = sales_df.groupBy("CustomerID") \
    .agg(sum("RevenueGenerated").alias("CustomerLifetimeValue"))

clv.show(10)

# Product Popularity Score: Weighted by total sales and revenue
product_popularity = sales_df.groupBy("StockCode", "Description") \
    .agg(
        sum("Quantity").alias("TotalUnitsSold"),
        sum("RevenueGenerated").alias("TotalRevenue")
    ) \
    .withColumn("ProductPopularityScore", col("TotalUnitsSold") * col("TotalRevenue"))

product_popularity.show(10)

# Seasonal Trends: Sales aggregated by month across all products
seasonal_trends_all = sales_df.groupBy("Month") \
    .agg(sum("Quantity").alias("TotalMonthlySales")) \
    .orderBy("Month")

seasonal_trends_all.show()

# Save results for further analysis
output_path = "output_sales_aggregation"
total_sales_per_product.write.csv(f"{output_path}/total_sales_per_product", header=True, mode="overwrite")
avg_revenue_per_country.write.csv(f"{output_path}/avg_revenue_per_country", header=True, mode="overwrite")
top_products.write.csv(f"{output_path}/top_products", header=True, mode="overwrite")
seasonal_trends.write.csv(f"{output_path}/seasonal_trends", header=True, mode="overwrite")
clv.write.csv(f"{output_path}/customer_lifetime_value", header=True, mode="overwrite")
product_popularity.write.csv(f"{output_path}/product_popularity", header=True, mode="overwrite")
seasonal_trends_all.write.csv(f"{output_path}/seasonal_trends_all", header=True, mode="overwrite")

print(f"Results saved to {output_path}")


24/12/06 04:03:28 WARN Utils: Your hostname, codespaces-3e6bbc resolves to a loopback address: 127.0.0.1; using 10.0.2.121 instead (on interface eth0)
24/12/06 04:03:28 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/12/06 04:03:28 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Initial dataset:
+---------+---------+--------------------+--------+------------+---------+----------+--------------+
|InvoiceNo|StockCode|         Description|Quantity| InvoiceDate|UnitPrice|CustomerID|       Country|
+---------+---------+--------------------+--------+------------+---------+----------+--------------+
|   536365|   85123A|WHITE HANGING HEA...|       6|12/1/10 8:26|     2.55|     17850|United Kingdom|
|   536365|    71053| WHITE METAL LANTERN|       6|12/1/10 8:26|     3.39|     17850|United Kingdom|
|   536365|   84406B|CREAM CUPID HEART...|       8|12/1/10 8:26|     2.75|     17850|United Kingdom|
|   536365|   84029G|KNITTED UNION FLA...|       6|12/1/10 8:26|     3.39|     17850|United Kingdom|
|   536365|   84029E|RED WOOLLY HOTTIE...|       6|12/1/10 8:26|     3.39|     17850|United Kingdom|
|   536365|    22752|SET 7 BABUSHKA NE...|       2|12/1/10 8:26|     7.65|     17850|United Kingdom|
|   536365|    21730|GLASS STAR FROSTE...|       6|12/1/10 8:26|     4.25|

24/12/06 04:03:40 WARN GarbageCollectionMetrics: To enable non-built-in garbage collector(s) List(G1 Concurrent GC), users should configure it(them) to spark.eventLog.gcMetrics.youngGenerationGarbageCollectors or spark.eventLog.gcMetrics.oldGenerationGarbageCollectors


+----+-----+---------+--------------------+-----------------+---------------------+
|Year|Month|StockCode|         Description|TotalQuantitySold|TotalRevenueGenerated|
+----+-----+---------+--------------------+-----------------+---------------------+
|NULL| NULL|    10002|INFLATABLE POLITI...|              860|    759.8900000000002|
|NULL| NULL|    10002|                NULL|              177|                  0.0|
|NULL| NULL|    10080|GROOVY CACTUS INF...|              303|   119.08999999999999|
|NULL| NULL|    10080|                NULL|              170|                  0.0|
|NULL| NULL|    10080|               check|               22|                  0.0|
|NULL| NULL|    10120|        DOGGY RUBBER|              193|   40.530000000000015|
|NULL| NULL|   10123C|                NULL|              -18|                  0.0|
|NULL| NULL|   10123C|HEARTS WRAPPING T...|                5|                 3.25|
|NULL| NULL|   10123G|                NULL|              -38|               

+------------------+--------------------+
|           Country|AvgRevenuePerCountry|
+------------------+--------------------+
|            Sweden|   79.21192640692637|
|         Singapore|    39.8270305676856|
|           Germany|   23.34894260136918|
|            France|  23.069288301975035|
|            Greece|   32.26383561643835|
|European Community|   21.17622950819672|
|           Belgium|   19.77330111164815|
|           Finland|  32.124805755395684|
|             Malta|   19.72811023622048|
|       Unspecified|  10.649753363228692|
|             Italy|  21.034259028642584|
|              EIRE|   32.12259882869682|
|         Lithuania|   47.45885714285714|
|            Norway|   32.37887661141805|
|             Spain|  21.624390051322607|
|           Denmark|  48.247146529562954|
|         Hong Kong|    35.1286111111111|
|           Iceland|  23.681318681318682|
|            Israel|  26.625656565656566|
|   Channel Islands|  26.499063324538245|
+------------------+--------------

...

+---------+--------------------+-----+------------+
|StockCode|         Description|Month|MonthlySales|
+---------+--------------------+-----+------------+
|    21212|PACK OF 72 RETROS...| NULL|       36039|
|    22197|      POPCORN HOLDER| NULL|       36334|
|    84077|WORLD WAR 2 GLIDE...| NULL|       53847|
|    84879|ASSORTED COLOUR B...| NULL|       36381|
|   85099B|JUMBO BAG RED RET...| NULL|       47363|
+---------+--------------------+-----+------------+

Feature engineering...


+----------+---------------------+
|CustomerID|CustomerLifetimeValue|
+----------+---------------------+
|     17420|    598.8300000000002|
|     16861|               151.65|
|     16503|   1421.4300000000003|
|     15727|    5178.959999999999|
|     17389|             31300.08|
|     15447|               155.17|
|     14450|               483.25|
|     13623|               672.44|
|     13285|   2709.1199999999994|
|     16339|   109.95000000000002|
+----------+---------------------+
only showing top 10 rows



+---------+--------------------+--------------+------------------+----------------------+
|StockCode|         Description|TotalUnitsSold|      TotalRevenue|ProductPopularityScore|
+---------+--------------------+--------------+------------------+----------------------+
|   84279P|CHERRY BLOSSOM  D...|           364|1516.9599999999998|             552173.44|
|    85015|SET OF 12  VINTAG...|          2182|1719.8400000000006|    3752690.8800000013|
|    21249|WOODLAND  HEIGHT ...|           724| 2160.000000000001|    1563840.0000000007|
|    21002|ROSE DU SUD DRAWS...|           125| 428.6600000000002|     53582.50000000002|
|    84987|SET OF 36 TEATIME...|          3038| 4452.320000000005|  1.3526148160000015E7|
|    20671|BLUE TEATIME PRIN...|            10|              12.5|                 125.0|
|    22690|DOORMAT HOME SWEE...|          1617| 9911.299999999996|  1.6026572099999992E7|
|    22708|     WRAP DOLLY GIRL|          2775|            1149.5|             3189862.5|
|   90184A

+-----+-----------------+
|Month|TotalMonthlySales|
+-----+-----------------+
| NULL|          5176450|
+-----+-----------------+



Results saved to output_sales_aggregation
